# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [2]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [3]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [18]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-01/'
water_dir = 'Water Sampling/'
ctd_dir = 'KN-214_CTD/'
leg = ''

In [19]:
os.listdir(basepath+array+cruise+water_dir)

['OOI Nov. 2013 DIC and TA data.eml',
 'OOIchl.xls',
 'OOI_2013_Nov_DIC and TA data.xlsx',
 'OOI_chl_labNB.pdf',
 'Pioneer Nutrient Lab sample data 2014.xlsx',
 'Pioneer-1_KN-214_Sampling_Log-written.pdf',
 'Pioneer-1_KN-214_Sampling_Log.xlsx',
 'Salts and O2']

In [10]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
btlpath = basepath+array+cruise+'KN-214_CTD/'
summary_sheet_path = basepath+array+cruise+'/Water Sampling/Pioneer-1_KN-214_Sampling_Log.xlsx'
salts_and_o2_path = basepath+array+cruise+'/Water Sampling/Salts and O2/'
nutrients_path = basepath+array+cruise+'/Water Sampling/Pioneer Nutrient Lab sample data 2014.xlsx'
chl_path = basepath+array+cruise+'/Water Sampling/'

In [ ]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [ ]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(btlpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(btlpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast #'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(btlpath+outname)

In [ ]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(btlpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(btlpath+file))
    else:
        pass

In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df.rename(columns={'Bottle Position':'Niskin #'},inplace=True)
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast #'] = df['Cast #'].apply(lambda x: str(x).zfill(3) )

### Oxygen & Salinity 
Now, we need to add the 

In [ ]:
def clean_sal_files(dirpath):
    
    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for files in os.listdir(dirpath):
            sample = []
            salinity = []
            with open(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+files) as file:
                data = file.readlines()
                for ind1,line in enumerate(data):
                    if ind1 == 0:
                        strs = data[0].replace('"','').split(',')
                        cruisename = strs[0]
                        station = strs[1]
                        cast = strs[2]
                        case = strs[8]
                    elif int(line.split()[0]) == 0:
                        pass
                    else:
                        strs = line.split()
                        sample.append(strs[0])
                        salinity.append(strs[2]) 
                # Generate a pandas dataframe to populate data
                data_dict = {'Cruise ID':cruisename,'Station #':station,'Cast #':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                df = pd.DataFrame.from_dict(data_dict)
                df.to_csv(file.name.replace('.','')+'.csv')
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.rename({'Cruise':'Cruise ID','Station':'Station #','Sample':'Sample ID','Salinity':'Salinity [psu]'},
          axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for file in os.listdir(dirpath):
        if 'OXY' in file:
            df = df.append(pd.read_excel(dirpath+file))
    # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Cruise':'Cruise ID','Station':'Station #','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
          axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
    df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise ID'] = df['Cruise ID'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

In [ ]:
dirpath = basepath+array+cruise+'Water Sampling/'
os.listdir(dirpath)

In [ ]:
file = 'Pioneer-1_KN-214_Sampling_Log.xlsx'
sample_log = pd.read_excel(dirpath+file,sheet_name='Summary',header=0)

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
os.listdir(basepath+array+cruise)

In [ ]:
sample_log.columns.values

In [ ]:
match = []
for x in list(sample_log.columns.values):
    y = 'Cast #'
    match.append(fuzz.token_set_ratio(x,y))
    

In [ ]:
match

In [ ]:
sample_log.head()

In [ ]:
# Rename the column headers
sample_log.rename(columns=lambda x: 'Sample Log: ' + x.strip(), inplace=True)
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: int(x.replace('*','')) if type(x) == str else x )
sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: x if np.isnan(x) else str( int(x) ) )
sample_log['Sample Log: Cast #'] = sample_log['Sample Log: Station-Cast #'].apply(lambda x: str(x).zfill(3))

In [ ]:
set(list(sample_log['Sample Log: Niskin #']))

### Nutrient Data

In [ ]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-01/'
leg = '/'

In [ ]:
os.listdir(basepath+array+cruise+'Water Sampling/')

In [ ]:
dirpath = basepath+array+cruise+'Water Sampling/'

In [ ]:
nutfile = [x for x in os.listdir(dirpath) if 'nutrient' in x.lower()]
chlfile = [x for x in os.listdir(dirpath) if 'chl' in x.lower() and x.endswith('.pdf') == False]
nutfile, chlfile

In [ ]:
nutrients = pd.read_excel(basepath+array+cruise+'Water Sampling/'+nutfile[0])

In [ ]:
nutrients

In [ ]:
chl = pd.read_excel(dirpath+chlfile[0])
chl.head()

In [ ]:
chl.columns.values

In [ ]:
chl[['Station-Cast #','Niskin #','Brown Bottle #','Chl (ug/l)','Phaeo (ug/l)']]

In [ ]:
sal_df.rename(columns=lambda x: 'SAL: ' + x.strip(), inplace=True)
sal_df.dropna(subset=['SAL: Cruise'], inplace=True)
sal_df.fillna(-999, inplace=True)
sal_df['SAL: Station'] = sal_df['SAL: Station'].apply(lambda x: str( int(x) ).zfill(3) )
sal_df['SAL: Niskin #'] = sal_df['SAL: Niskin #'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Sample'].apply(lambda x: str( int(x) ) )
sal_df['SAL: Sample'] = sal_df['SAL: Case ID'] + sal_df['SAL: Sample']

In [ ]:
oxy_df.rename(columns=lambda x: 'OXY: ' + x.strip(), inplace=True)
oxy_df.dropna(subset=['OXY: Cruise'], inplace=True)
oxy_df.fillna(-999, inplace=True)
oxy_df['OXY: Station'] = oxy_df['OXY: Station'].apply(lambda x: str( int(x) ).zfill(3) )
oxy_df['OXY: Niskin #'] = oxy_df['OXY: Niskin #'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Sample#'].apply(lambda x: str( int(x) ) )
oxy_df['OXY: Sample#'] = oxy_df['OXY: Case'] + oxy_df['OXY: Sample#']

In [ ]:
# Now I need to merge dataframes together
sample_log = sample_log.merge(sal_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Salts Bottle #'], right_on=['SAL: Station','SAL: Sample'])

In [ ]:
sample_log = sample_log.merge(oxy_df, how='left', left_on=['Sample Log: Cast #','Sample Log: Oxygen Bottle #'],
                              right_on=['OXY: Station','OXY: Sample#'])

In [ ]:
os.listdir('Irminger/Irminger-5/Water Sampling/')

In [ ]:
nutrients = pd.read_excel(nutrients_path,header=0)

In [ ]:
nutrient_bottles = sample_log['Sample Log: Nitrate Bottle 1'].str.split(',').apply(pd.Series, 1).stack()
nutrient_bottles.index = nutrient_bottles.index.droplevel(-1)
nutrient_bottles.name = 'Nitrate Bottle #'

In [ ]:
nutrient_bottles

In [ ]:
# Add the nutrient bottle number back into the sample log, and remove the excess '.'
sample_log = sample_log.join(nutrient_bottles)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace('.','') if type(x) == str else x)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace(' ','') if type(x) == str else x)

In [ ]:
# Now I can add the nutrient bottle data to the sample log before loading
sample_log = sample_log.merge(nutrients, how='left', left_on='Nitrate Bottle #', right_on='Sample ID')

In [ ]:
sample_log.columns.values

In [ ]:
sample_columns = ['Sample Log: Cast #','Sample Log: Niskin #','SAL: Salinity','OXY: Oxy','Avg: Nitrate+Nitrite [µmol/L]', 'Avg: Ammonium [µmol/L]',
    'Avg: Phosphate [µmol/L]', 'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]', 'Avg: Nitrate [µmol/L]']

In [ ]:
result = df.merge(sample_log[sample_columns], how='left',
                  left_on=['Cast #','Niskin #'], right_on=['Sample Log: Cast #','Sample Log: Niskin #'])

In [ ]:
result

In [ ]:
result.columns.values

In [ ]:
sal_unit = '[' + list(set(sal_df['SAL: Unit']) )[0] + ']'
oxy_unit = '[' + list(set(oxy_df['OXY: Unit']) )[0] + ']'
sal_unit, oxy_unit

In [ ]:
result.drop(['Sample Log: Cast #','Sample Log: Niskin #'], axis=1, inplace=True)

In [ ]:
result.rename(columns=lambda x: x.replace('SAL:','Bottle') + ' ' + sal_unit if 'SAL:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('OXY:','Bottle') + ' ' + oxy_unit if 'OXY:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('Avg:','Bottle') if 'Avg:' in x else x, inplace=True)

In [ ]:
result

In [ ]:
# Replace all of the nans with -999999 and save to a csv
result.fillna(str(-9999999),inplace=True)

In [ ]:
result

In [ ]:
result.to_csv(salts_and_o2_path+'Irminger-3_Summary.csv')

In [ ]:
os.getcwd()